![](../docs/banner.png)

# L5: Narzędzia zarządzania eksperymentami

## Efekty kształcenia laboratorium
<hr>

- zapoznasz się z zasadami BHP
- dowiesz się, co będziemy ćwiczyć w trakcie tego semestru
- poznasz zasady oceniania 
- poznasz możliwości tworzenia środowiska wirtualnego Python
- poznasz środowisko Jupyter
- poznasz podstawowe mechanizmy systemu kontroli wersji Git i ekosystemu GitHub